In [73]:
import os
import warnings
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import accuracy_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline

import optuna
from optuna import Trial
from optuna.samplers import TPESampler
from optuna.pruners import MedianPruner

from lightgbm import LGBMClassifier

SEED = 42 # Muito importante manter a SEED igual em todos os modelos para garantir a consistência dos dados no ensemble
FOLDS = 5 # Muito importante manter o mesmo número de FOLDS em todos os modelos para garantir a consistência dos dados no ensemble

In [74]:
sintetico = pd.read_csv('../../src/train/train.csv', index_col='id')
original = pd.read_csv('../../src/train/original.csv')
test = pd.read_csv('../../src/test/test.csv', index_col='id')

train = pd.concat([sintetico, original], ignore_index=True)

initial_features = list(test.columns)

In [75]:
def cross_validation(model, X, y, encoder, scoring=accuracy_score, k=FOLDS):

    skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=SEED)

    scores = []
    out_of_fold = []
    
    for i, (train_index, val_index) in enumerate(skf.split(X, y)):
        print(f"Fold {i + 1}")
        
        X_train = X.iloc[train_index]
        y_train = y.iloc[train_index]

        X_val = X.iloc[val_index]
        y_val = y.iloc[val_index]
        
        model.fit(X_train, y_train)
        probabilidades = model.predict_proba(X_val)

        # Recuperar a predição final a partir das probabilidades
        indices_predicoes = np.argmax(probabilidades, axis=1)
        classes_preditas = model.classes_[indices_predicoes]

        score = scoring(y_val, classes_preditas)

        scores.append(score)

        true_label = pd.Series(y_val.values, name='true')

        pred_label_df = pd.DataFrame(probabilidades)

        oof_pred = pd.concat([pred_label_df, true_label], axis=1, ignore_index=True)
        oof_pred.columns = [f'pred_{encoder[model.classes_[0]]}', f'pred_{encoder[model.classes_[1]]}', f'pred_{encoder[model.classes_[2]]}', 'true']

        out_of_fold.append(oof_pred)

    print(f"Score: {np.mean(scores)}")
    
    return scores, out_of_fold

In [76]:
def save_oof(oof):

    os.makedirs('oof', exist_ok=True)

    for i, fold in enumerate(oof):
        fold.to_csv(f'oof/fold_{i+1}.csv', index=False)

In [77]:
def predict_test(model, X_train, y_train, X_test, encoder):

    model.fit(X_train, y_train)

    probabilidades = model.predict_proba(X_test)
    pred_label_df = pd.DataFrame(probabilidades)

    pred_label_df.columns = [f'pred_{encoder[model.classes_[0]]}', f'pred_{encoder[model.classes_[1]]}', f'pred_{encoder[model.classes_[2]]}']

    os.makedirs('test', exist_ok=True)

    pred_label_df.to_csv(f'test/test_pred.csv', index=False)

    return pred_label_df

In [78]:
y = train['Target'].replace({"Enrolled": 0, "Graduate": 1, "Dropout": 2})
X = train[initial_features]

In [79]:
# threshold = 15

# cat_features = []

# for col in initial_features:
#     unique_values = train[col].nunique()
    
#     if unique_values <= threshold:
#         cat_features.append(col)

# for feature in cat_features:
#     for df in [train, test]:
#         df[feature] = df[feature].astype('category')


num_features = X.select_dtypes(include=['int64', 'float64']).columns

In [80]:
encoder = {0: "Enrolled", 1: "Graduate", 2: "Dropout"}

In [81]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_features),
    ])

lgbm = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LGBMClassifier(random_state = 42, verbose=-1,
                                    max_depth= 8, 
                                    learning_rate= 0.00694099927384355, 
                                    n_estimators = 1689,
                                    min_child_weight = 0.22269847051511316
                                )
    )
])

In [82]:
scores,oof = cross_validation(lgbm, X, y, encoder)

Fold 1
Fold 2
Fold 3
Fold 4
Fold 5
Score: 0.8300264185339831


In [83]:
for fold in oof:
    fold['true'] = fold['true'].replace(encoder)
    
save_oof(oof)

In [94]:
# Função objetivo para a otimização
def objective(trial: Trial, X_cp, y):

    X = X_cp.copy()

    param = {
        'objective': 'multiclass',
        'metric': 'multi_logloss',
        'verbosity': -1,
        'boosting_type': trial.suggest_categorical('boosting_type', ['gbdt', 'dart', 'goss']),
        'num_class': 3,  # Número de classes
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'max_depth': trial.suggest_int('max_depth', -1, 64),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-8, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 10.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 10.0)
    }

    # Adicionar parâmetros específicos para 'dart'
    if param['boosting_type'] == 'dart':
        param['drop_rate'] = trial.suggest_uniform('drop_rate', 0.1, 0.5)
        param['skip_drop'] = trial.suggest_uniform('skip_drop', 0.5, 1.0)
    
    # Adicionar parâmetros específicos para 'goss'
    if param['boosting_type'] == 'goss':
        param['top_rate'] = trial.suggest_uniform('top_rate', 0.0, 0.5)
        param['other_rate'] = trial.suggest_uniform('other_rate', 0.0, 0.5)

    threshold = trial.suggest_int('threshold', 1, 100)

    cat_features = []

    for col in initial_features:
        unique_values = X[col].nunique()
        
        if unique_values <= threshold:
            X[col] = X[col].astype('category')
            cat_features.append(col)

    preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_features),
    ])

    lgbm = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LGBMClassifier(**param))
                    ])
    
    scores,_ = cross_validation(lgbm, X, y, encoder, k=3)
    accuracy = np.mean(scores)


    return accuracy

In [95]:
warnings.filterwarnings("ignore")

# Configuração do estudo
study = optuna.create_study(direction='maximize', sampler=TPESampler(), pruner=MedianPruner())
study.optimize(lambda trial: objective(trial, X, y), n_trials=1000)

[I 2024-06-24 20:31:41,281] A new study created in memory with name: no-name-63a5a075-f1ce-474c-98ac-04846695cd28


Fold 1
Fold 2
Fold 3


[I 2024-06-24 20:34:35,719] Trial 0 finished with value: 0.475538039576432 and parameters: {'boosting_type': 'goss', 'num_leaves': 168, 'max_depth': 24, 'learning_rate': 5.6269388230625705e-06, 'n_estimators': 805, 'min_child_samples': 85, 'subsample': 0.8441427652522626, 'colsample_bytree': 0.9478659480174133, 'reg_alpha': 0.25979613937882073, 'reg_lambda': 1.5071219774536052e-05, 'top_rate': 0.4851252977690672, 'other_rate': 0.36043383414069824, 'threshold': 38}. Best is trial 0 with value: 0.475538039576432.


Score: 0.475538039576432
Fold 1
Fold 2
Fold 3


[I 2024-06-24 20:46:52,392] Trial 1 finished with value: 0.475538039576432 and parameters: {'boosting_type': 'dart', 'num_leaves': 248, 'max_depth': 27, 'learning_rate': 1.5752580863232077e-05, 'n_estimators': 481, 'min_child_samples': 94, 'subsample': 0.9460968012186644, 'colsample_bytree': 0.8664482249338481, 'reg_alpha': 5.517378246196934e-06, 'reg_lambda': 0.0003478251326441819, 'drop_rate': 0.2337887951543568, 'skip_drop': 0.5180463685385117, 'threshold': 49}. Best is trial 0 with value: 0.475538039576432.


...

Score: 0.82688840732783
Fold 1
Fold 2
Fold 3


[I 2024-06-25 09:13:39,524] Trial 998 finished with value: 0.8308912726631422 and parameters: {'boosting_type': 'goss', 'num_leaves': 213, 'max_depth': 34, 'learning_rate': 0.018496301261093834, 'n_estimators': 1000, 'min_child_samples': 43, 'subsample': 0.8165499429276507, 'colsample_bytree': 0.6684294610197853, 'reg_alpha': 4.0954689945015406e-07, 'reg_lambda': 7.017430700144118e-06, 'top_rate': 0.27676433205038653, 'other_rate': 0.4334913313635946, 'threshold': 63}. Best is trial 533 with value: 0.8334239392413227.


Score: 0.8308912726631422
Fold 1
Fold 2
Fold 3


[I 2024-06-25 09:14:06,233] Trial 999 finished with value: 0.8331150904524628 and parameters: {'boosting_type': 'goss', 'num_leaves': 23, 'max_depth': -1, 'learning_rate': 0.03386318617035505, 'n_estimators': 969, 'min_child_samples': 49, 'subsample': 0.915924662900106, 'colsample_bytree': 0.5672041585070235, 'reg_alpha': 8.626112538054128e-07, 'reg_lambda': 1.9141248094844187e-08, 'top_rate': 0.05929489277294197, 'other_rate': 0.4543767621844024, 'threshold': 58}. Best is trial 533 with value: 0.8334239392413227.


Score: 0.8331150904524628


In [96]:
# Melhor conjunto de hiperparâmetros
print(f"Best trial: {study.best_trial.number}")
print(f"Best value: {study.best_value}")
print(f"Best params: {study.best_trial.params}")

Best trial: 533
Best value: 0.8334239392413227
Best params: {'boosting_type': 'goss', 'num_leaves': 26, 'max_depth': 29, 'learning_rate': 0.019777145894017006, 'n_estimators': 961, 'min_child_samples': 60, 'subsample': 0.9206395466786965, 'colsample_bytree': 0.6516576146105851, 'reg_alpha': 3.564276489131016e-08, 'reg_lambda': 0.004800009488178458, 'top_rate': 0.025957543189225753, 'other_rate': 0.47922731376330013, 'threshold': 49}


In [97]:
best_threshold = 49

        
lgbm_best_params = {'objective': 'multiclass', 'metric': 'multi_logloss','num_class': 3,'boosting_type': 'goss', 'num_leaves': 26, 'max_depth': 29, 'learning_rate': 0.019777145894017006, 'n_estimators': 961, 'min_child_samples': 60, 'subsample': 0.9206395466786965, 'colsample_bytree': 0.6516576146105851, 'reg_alpha': 3.564276489131016e-08, 'reg_lambda': 0.004800009488178458, 'top_rate': 0.025957543189225753, 'other_rate': 0.47922731376330013, 'verbosity': -1,}


In [98]:
cat_features = []

for col in initial_features:
    unique_values = X[col].nunique()
    
    if unique_values <= best_threshold:
        X[col] = X[col].astype('category')
        cat_features.append(col)

preprocessor = ColumnTransformer(
transformers=[
    ('num', StandardScaler(), num_features),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_features),
])

lgbm = Pipeline(steps=[
('preprocessor', preprocessor),
('classifier', LGBMClassifier(**lgbm_best_params))
                ])

In [99]:
scores,oof = cross_validation(lgbm, X, y, encoder)

Fold 1
Fold 2
Fold 3
Fold 4
Fold 5
Score: 0.8326826338792704


In [100]:
scores

[0.8348261164988573,
 0.8327876953486936,
 0.8303063998023227,
 0.83117123795404,
 0.8343217197924389]

In [101]:
for fold in oof:
    fold['true'] = fold['true'].replace(encoder)

save_oof(oof)

In [102]:
X_test = test[initial_features]

predict_test(lgbm, X, y, X_test, encoder)

,pred_Enrolled,pred_Graduate,pred_Dropout
0,0.003152,0.002334,0.994514
1,0.011473,0.984897,0.003630
2,0.229215,0.735398,0.035387
3,0.430074,0.341587,0.228339
4,0.640338,0.049929,0.309733
...,...,...,...
51007,0.053626,0.088670,0.857704
51008,0.019440,0.000473,0.980087
51009,0.011323,0.007578,0.981100
51010,0.068406,0.014222,0.917372
